In [ ]:
import os

directories = ['feature_vectors_iv3','feature_vectors_iv3/train','feature_vectors_iv3/test','feature_vectors_iv3/validation']

for dir in directories:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
from PIL import Image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

class FeatureExtractor:
    def __init__(self):
        self.model = InceptionV3(weights='imagenet')
        #self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

    def extract(self, img):
        """
        Extract a deep feature from an input image
        Args:
            img: from PIL.Image.open(path) or tensorflow.keras.preprocessing.image.load_img(path)

        Returns:
            feature (np.ndarray): deep feature with the shape=(4096, )
        """
        img = img.resize((299, 299))  # VGG must take a 224x224 img as an input
        img = img.convert('RGB')  # Make sure img is color
        x = image.img_to_array(img)  # To np.array. Height x Width x Channel. dtype=float32
        x = np.expand_dims(x, axis=0)  # (H, W, C)->(1, H, W, C), where the first elem is the number of img
        x = preprocess_input(x)  # Subtracting avg values for each pixel
        feature = self.model.predict(x)[0]  # (1, 4096) -> (4096, )
        return feature / np.linalg.norm(feature)  # Normalize



In [ ]:
from pathlib import Path
fe = FeatureExtractor()

i = 0
for img_path in sorted(Path("../webscraping/images_aggregate").glob("*.jpg")):
    i+=1
    print(img_path)  
    feature = fe.extract(img=Image.open(img_path))
    
    if(i%5==0 and i%10!=0):
        feature_path = Path("./feature_vectors_iv3/test") / (img_path.stem + ".npy")  # e.g., ./static/feature/xxx.npy
        np.save(feature_path, feature)
    elif(i%10==0):
        feature_path = Path("./feature_vectors_iv3/validation") / (img_path.stem + ".npy")  # e.g., ./static/feature/xxx.npy
        np.save(feature_path, feature)
    else:
        feature_path = Path("./feature_vectors_iv3/train") / (img_path.stem + ".npy")  
        np.save(feature_path, feature)